In [30]:
import pandas as pd
import os

In [31]:
df = pd.read_json("annotated_data.json", orient="records")

In [32]:
df.head(10)

,species_file,binomial_name,paragraph_index,paragraph_class,paragraph_file
0,5384359.json,Amaranthus powellii,0,n,5384359_000.txt
1,5384359.json,Amaranthus powellii,1,y,5384359_001.txt
2,5384359.json,Amaranthus powellii,2,n,5384359_002.txt
3,5384359.json,Amaranthus powellii,3,n,5384359_003.txt
4,5954949.json,Boletus pinophilus,0,n,5954949_000.txt
5,5954949.json,Boletus pinophilus,1,y,5954949_001.txt
6,5954949.json,Boletus pinophilus,2,n,5954949_002.txt
7,5954949.json,Boletus pinophilus,3,n,5954949_003.txt
8,5954949.json,Boletus pinophilus,4,n,5954949_004.txt
9,5954949.json,Boletus pinophilus,5,n,5954949_005.txt


In [33]:
df.paragraph_class.value_counts()

n    739
y    110
Name: paragraph_class, dtype: int64

In [34]:
## split
positives = df.loc[df["paragraph_class"]=="y"]
positives_val = positives.sample(10, random_state=42)
positives_val["split"] = "val"
positives_train = positives.drop(positives_val.index)
positives_train["split"] = "train"

negatives = df.loc[df["paragraph_class"]=="n"]
negatives_val = negatives.sample(10, random_state=42)
negatives_val["split"] = "val"
negatives_train = negatives.drop(negatives_val.index)
negatives_train["split"] = "train"

df = pd.concat([positives_train,positives_val,negatives_train,negatives_val])

In [35]:
species_path = "/data/nicola/WSH/final_data/species/"
species_list = os.listdir(species_path)
## Species used in train, test and/or val sets
used_species = pd.read_json("/data/nicola/WSH/final_data/L2_species_keys.json", orient="records")["species_key"].tolist()
used_species = [str(spe)+".json" for spe in used_species]
## Species to be used for annotation
unused_species = [spe for spe in species_list if spe not in used_species]
annotated_data = pd.read_json("./annotated_data.json", orient="records")
#annotated_data = pd.DataFrame(columns=["species_file","binomial_name","paragraph_index","paragraph_class","paragraphe_file"])
## Remove previously annotated files
#unused_species = [spe for spe in unused_species if spe not in annotated_data["species_file"].unique().tolist()]

In [36]:
unused_species_ids = [int(spe[:-5])for spe in unused_species]
unused_species_ids[10]

6366886

In [37]:
gbif_raw = pd.read_csv("/data/nicola/WSH/raw_data/gbif_raw.csv", sep="\t")

/tmp/ipykernel_1634385/3341368087.py:1: DtypeWarning: Columns (10,14,16,17,36,37,38,39,40,41,43,44,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  gbif_raw = pd.read_csv("/data/nicola/WSH/raw_data/gbif_raw.csv", sep="\t")


In [38]:
gbif_raw.columns

Index(['gbifID', 'datasetKey', 'occurrenceID', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'species', 'infraspecificEpithet',
       'taxonRank', 'scientificName', 'verbatimScientificName',
       'verbatimScientificNameAuthorship', 'countryCode', 'locality',
       'stateProvince', 'occurrenceStatus', 'individualCount',
       'publishingOrgKey', 'decimalLatitude', 'decimalLongitude',
       'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation',
       'elevationAccuracy', 'depth', 'depthAccuracy', 'eventDate', 'day',
       'month', 'year', 'taxonKey', 'speciesKey', 'basisOfRecord',
       'institutionCode', 'collectionCode', 'catalogNumber', 'recordNumber',
       'identifiedBy', 'dateIdentified', 'license', 'rightsHolder',
       'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted',
       'mediaType', 'issue'],
      dtype='object')

In [39]:
gbif_subset = gbif_raw.loc[gbif_raw["speciesKey"].isin(unused_species_ids)]
gbif_subset = gbif_subset[["speciesKey","kingdom","phylum"]].drop_duplicates()
gbif_subset["speciesKey"] = gbif_subset["speciesKey"].apply(lambda x : int(x))

In [40]:
gbif_subset.loc[gbif_subset["phylum"]=="Chordata"]

,speciesKey,kingdom,phylum
194,2359706,Animalia,Chordata
550,2498112,Animalia,Chordata
564,2480259,Animalia,Chordata
1030,2432614,Animalia,Chordata
1109,2432427,Animalia,Chordata
...,...,...,...
4470440,4287131,Animalia,Chordata
4470453,5204862,Animalia,Chordata
4610568,2351630,Animalia,Chordata
4741062,2432509,Animalia,Chordata


In [41]:
gbif_subset.set_index("speciesKey")

,kingdom,phylum
speciesKey,,
1633937,Animalia,Arthropoda
5107829,Animalia,Arthropoda
3227657,Bacteria,Firmicutes
5073713,Animalia,Arthropoda
7907958,Animalia,Arthropoda
...,...,...
2014676,Animalia,Arthropoda
5153706,Animalia,Arthropoda
1831643,Animalia,Arthropoda


In [42]:
annotated_data = pd.read_json("./annotated_data.json", orient="records")
annotated_data["species_key"] = annotated_data["species_file"].apply(lambda x : int(x[:-5]))
annotated_data = annotated_data.join(gbif_subset.set_index("speciesKey"), on="species_key")

In [43]:
annotated_data[["species_key","kingdom","phylum"]].drop_duplicates()["kingdom"].value_counts()

Animalia    47
Plantae     44
Fungi       39
Bacteria     3
Name: kingdom, dtype: int64

In [44]:
annotated_data[["species_key","kingdom","phylum"]].drop_duplicates()["phylum"].value_counts()

Tracheophyta        38
Arthropoda          33
Basidiomycota       22
Ascomycota          17
Chordata            11
Bryophyta            5
Mollusca             3
Proteobacteria       1
Firmicutes_A         1
Actinobacteriota     1
Chlorophyta          1
Name: phylum, dtype: int64

In [45]:
len(unused_species_ids)

3020

In [46]:
annotated_data[annotated_data["species_key"]==4470453]

,species_file,binomial_name,paragraph_index,paragraph_class,paragraph_file,species_key,kingdom,phylum
